In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas
import os
import pingouin
import datapungibea as dpb
from matplotlib.patches import ArrowStyle
import copy
from matplotlib.backends.backend_pdf import PdfPages
import networkx as nx
from pgmpy.estimators import PC
import statsmodels.api as sm
import plotly.express as px
from sklearn.linear_model import LinearRegression
import seaborn as sns
import random
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.metrics import mean_squared_error, accuracy_score
from statsmodels.discrete.discrete_model import Logit
from networkx.drawing.nx_agraph import graphviz_layout
from collections import defaultdict
import math


C:\Users\abiodun.idowu\Anaconda3\envs\python37\lib\site-packages\geopandas\_compat.py:115: UserWarning: The Shapely GEOS version (3.4.3-CAPI-1.8.3 r4285) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string
C:\Users\abiodun.idowu\Anaconda3\envs\python37\lib\site-packages\outdated\utils.py:18: OutdatedPackageWarning: The package outdated is out of date. Your version is 0.2.1, the latest is 0.2.2.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  **kwargs
C:\Users\abiodun.idowu\Anaconda3\envs\python37\lib\site-packages\outdated\utils.py:18: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.2, the latest is 0.5.4.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  **kwargs
C:\Users\abiodun.idowu\Anaconda3\envs\python37\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Ple

In [2]:
from abbeyFunctions import graph_DAG, create_cluster_dags, determine_p_value, graph_DAG_cluster, get_edges_dict, get_col_shared_edges, get_row_shared_edges, plot_shared_col_edges, plot_shared_row_edges

In [3]:
ai_data_nominal = pd.read_csv('nominal_data.csv')

In [4]:
ai_data_real = pd.read_csv('real_data.csv')

In [5]:
ai_data_nominal = ai_data_nominal.set_index(['FIPS_y', 'TimePeriod'])

In [6]:
ai_data_real = ai_data_real.set_index(['FIPS_y', 'TimePeriod'])

In [7]:
exclude_vars = ['M4 I','RM4I', 'unem']  # List of variables to exclude

# Create a dictionary to store the dataframes
ai_dct = {"logged_n": ai_data_nominal.copy(), "logged_r": ai_data_real.copy(), 
          "rates_n": ai_data_nominal.copy(), "rates_r": ai_data_real.copy(),
         "percent rates_n": ai_data_nominal.copy(), 
          "percent rates_r": ai_data_real.copy()}

        
for col in ai_dct['logged_n'].columns:
    if col not in exclude_vars:
        ai_dct["logged_n"][col] = np.log(ai_data_nominal[col]).replace([np.inf, -np.inf], np.nan)

for col in ai_dct['logged_r'].columns:
    if col not in exclude_vars:
        ai_dct["logged_r"][col] = np.log(ai_data_real[col]).replace([np.inf, -np.inf], np.nan)


# Calculate rate of change (differences of log-transformed values)
for col in ai_dct['rates_n'].columns:
    if col not in exclude_vars:
        ai_dct["rates_n"][col] = ai_dct["logged_n"].groupby('FIPS_y')[col].diff().dropna()
        
        

for col in ai_dct['rates_r'].columns:
    if col not in exclude_vars:
        ai_dct["rates_r"][col] = ai_dct["logged_r"].groupby('FIPS_y')[col].diff().dropna()
        
        
        # Calculate percent rate of change
for col in ai_dct['percent rates_n'].columns:
    #if col not in exclude_vars:
        ai_dct["percent rates_n"][col] = ai_dct["rates_n"].groupby('FIPS_y')[col].diff().dropna()

        
        
for col in ai_dct['percent rates_r'].columns:
    #if col not in exclude_vars:
        ai_dct["percent rates_r"][col] = ai_dct["rates_r"].groupby('FIPS_y')[col].diff().dropna()

        
        

C:\Users\abiodun.idowu\Anaconda3\envs\python37\lib\site-packages\pandas\core\arraylike.py:364: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\abiodun.idowu\Anaconda3\envs\python37\lib\site-packages\pandas\core\arraylike.py:364: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [8]:
nominal = ai_dct['rates_n']

In [9]:
nominal.drop(columns=['All ', 'Neig'], inplace=True)

In [10]:
diff_nominal = ai_dct['percent rates_n']

In [11]:
diff_nominal.drop(columns=['All ', 'Neig'], inplace=True)

In [12]:
real = ai_dct['rates_r']

In [13]:
real.drop(columns=['All ', 'Neig'], inplace=True)

In [14]:
diff_real = ai_dct['percent rates_r']

In [15]:
diff_real.drop(columns=['All ', 'Neig'], inplace=True)

In [16]:
# nominal.to_csv('nominal.csv', index=True)
# diff_nominal.to_csv('diff_nominal.csv', index=True)
# real.to_csv('real.csv', index=True)
# diff_real.to_csv('diff_real.csv', index=True)

In [17]:
ai_df_nom = pd.read_csv('nominal_kml3d.csv')

ai_df_diffnom = pd.read_csv('diff_nominal_kml3d.csv')

ai_df_real = pd.read_csv('real_kml3d.csv')

ai_df_diffreal = pd.read_csv('diff_real_kml3d.csv')

In [18]:
level_11 = ai_df_nom

level_12 = ai_df_diffnom

level_21 = ai_df_real

level_22 = ai_df_diffreal

In [19]:
dfg_ai11 = nominal.dropna()

dfg_ai12 = diff_nominal.dropna()

dfg_ai21 = real.dropna()

dfg_ai22 = diff_real.dropna()

In [20]:
ai_merge11 = pd.merge(dfg_ai11, level_11[['FIPS_y', 'clusters']], on='FIPS_y', how='left')

In [21]:
ai_merge12 = pd.merge(dfg_ai12, level_12[['FIPS_y', 'clusters']], on='FIPS_y', how='left')

In [22]:
ai_merge21 = pd.merge(dfg_ai21, level_21[['FIPS_y', 'clusters']], on='FIPS_y', how='left')

In [23]:
ai_merge22 = pd.merge(dfg_ai22, level_22[['FIPS_y', 'clusters']], on='FIPS_y', how='left')

In [24]:
kml3d_ai_level11 = ai_merge11.dropna()

In [25]:
kml3d_ai_level12 = ai_merge12.dropna()

In [26]:
kml3d_ai_level21 = ai_merge21.dropna()

In [27]:
kml3d_ai_level22 = ai_merge22.dropna()

In [28]:
kml3d_ai_level11 = kml3d_ai_level11.set_index(['FIPS_y'])

In [29]:
kml3d_ai_level12 = kml3d_ai_level12.set_index(['FIPS_y'])

In [30]:
kml3d_ai_level21 = kml3d_ai_level21.set_index(['FIPS_y'])

In [31]:
kml3d_ai_level22 = kml3d_ai_level22.set_index(['FIPS_y'])

In [32]:
cluster_mapping = {'A': 1, 'B': 2, 'C': 3, 'D': 4}

In [33]:
kml3d_ai_level11['clusters'].replace(cluster_mapping, inplace=True)

In [34]:
kml3d_ai_level12['clusters'].replace(cluster_mapping, inplace=True)

In [35]:
kml3d_ai_level21['clusters'].replace(cluster_mapping, inplace=True)

In [36]:
kml3d_ai_level22['clusters'].replace(cluster_mapping, inplace=True)

In [37]:
nr_11 = kml3d_ai_level11
ndr_13 = kml3d_ai_level12
rr_21 = kml3d_ai_level21
rdr_23 = kml3d_ai_level22

In [38]:
nr_12 = nr_11.copy()
ndr_14 = ndr_13.copy()
rr_22 = rr_21.copy()
rdr_24 = rdr_23.copy()

In [39]:
rr_22.columns

Index(['Agri', 'Mini', 'Util', 'RM4_', 'RM4I', 'unem', 'clusters'], dtype='object')

In [40]:
nr_12.drop(columns=['M4_y', 'M4 I'], inplace=True)
ndr_14.drop(columns=['M4_y', 'M4 I'], inplace=True)

rr_22.drop(columns=['RM4_', 'RM4I'], inplace=True)
rdr_24.drop(columns=['RM4_', 'RM4I'], inplace=True)

In [41]:
nr_15 = nr_11.copy()
ndr_16 = ndr_13.copy()

rr_25 = rr_21.copy()
rdr_26 = rdr_23.copy()

In [42]:
rdr_26.columns

Index(['Agri', 'Mini', 'Util', 'RM4_', 'RM4I', 'unem', 'clusters'], dtype='object')

In [43]:
nr_15.drop(columns=['unem'], inplace=True)
ndr_16.drop(columns=['unem'], inplace=True)
rr_25.drop(columns=['unem'], inplace=True)
rdr_26.drop(columns=['unem'], inplace=True)

In [44]:
plot_df2 = nominal.copy()
plot_df3 = nominal.copy()
plot_df5 = diff_nominal.copy()
plot_df6 = diff_nominal.copy()
plot_df8 = real.copy()
plot_df9 = real.copy()
plot_df11 = diff_real.copy()
plot_df12 = diff_real.copy()

In [45]:
plot_df1 = nominal.dropna()
plot_df2.drop(columns=['M4_y', 'M4 I'], inplace=True)
plot_df3.drop(columns=['unem'], inplace=True)

plot_df4 = diff_nominal.dropna()
plot_df5.drop(columns=['M4_y', 'M4 I'], inplace=True)
plot_df6.drop(columns=['unem'], inplace=True)

plot_df7 = real.dropna()
plot_df8.drop(columns=['RM4_', 'RM4I'], inplace=True)
plot_df9.drop(columns=['unem'], inplace=True)

plot_df10 = diff_real.dropna()
plot_df11.drop(columns=['RM4_', 'RM4I'], inplace=True)
plot_df12.drop(columns=['unem'], inplace=True)

In [46]:
plot_df1 = plot_df1.dropna()
plot_df2 = plot_df2.dropna()
plot_df3 = plot_df3.dropna()
plot_df4 = plot_df4.dropna()
plot_df5 = plot_df5.dropna()
plot_df6 = plot_df6.dropna()
plot_df7 = plot_df7.dropna()
plot_df8 = plot_df8.dropna()
plot_df9 = plot_df9.dropna()
plot_df10 = plot_df10.dropna()
plot_df11 = plot_df11.dropna()
plot_df12 = plot_df12.dropna()

### DAGs representing nominal rates and real rates:

Where;

nr_11 represents nominal rates with all variables (except neighbor's GDP and total GDP),

nr_12 nominal rates (without macroeconomic variables)

nr_15 nominal rates (without unemployment rates)

rr_21 represents real rates with all variables (except neighbor's GDP and total GDP), and

rr_22 real rates (without macroeconomic variables)

rr_25 represents real rates without unemployment rates

### DAGs representing first difference rates

Where;

ndr_13 represents differenced nominal rates with all variables (except neighbor and total GDP),

ndr_14 differenced nominal rates (without macro variables)

ndr_16 differenced nominal rates (witout unemployment rates)

rr_23 represents differenced real rates with all variables (except neighbor and total GDP), and

rr_24 represents differenced real rates without M4 and interest rates and without neighbor and total GDP

rr_24 differenced real rates without unemployment rates

In [47]:
def all_results(dfs, name, folder, show=True):
    try:
        os.mkdir(folder)
    except FileExistsError:
        pass

    # Specify the absolute path for the output PDF file
    pdf_file_path = f"{folder}/{name}.pdf"
    pp = PdfPages(pdf_file_path)

    # Create a grid for the subplots
    fig, axes = plt.subplots(4, len(dfs), figsize=(5*len(dfs), 30))

    # Different PC algorithms to test
    algorithms = ["orig"]

    # Iterate over each dataframe
    for idx, dataframe in enumerate(dfs):
        # Get clustered dataframes for the current dataframe
        clustered_dfs = create_cluster_dags(dataframe)

        # Iterate over clustered dataframes and determine p-values dynamically
        for cluster_idx, cluster_df in enumerate(clustered_dfs):
            n = cluster_df.shape[0]  # Number of observations for the current cluster
            p_value = determine_p_value(n)

            # Iterate over different PC algorithms
            for algorithm in algorithms:
                c = PC(cluster_df)
                model = c.estimate(return_type='pdag', variant=algorithm, significance_level=p_value, ci_test='pearsonr')
                edges = model.edges

                # Create DAG for the current cluster, p-value, and algorithm using the modified graph_DAG function
                graph_DAG_cluster(edges, cluster_df, title=f'Dataframe {idx + 1} - Cluster {cluster_idx + 1} - p_value = {p_value}\nnum_observations={n}'
                          , algorithm=algorithm, ax=axes[cluster_idx, idx])

    # Save the figure to the PDF
    pp.savefig(fig, bbox_inches='tight')

    plt.close()    

    if show:
        plt.show()

        
    # Create and save plots for shared edges
    edges_dct = get_edges_dict(dfs)
    col_shared_edges=get_col_shared_edges(edges_dct)
    row_shared_edges=get_row_shared_edges(edges_dct)
    plot_shared_col_edges(col_shared_edges, name, folder)
    plot_shared_row_edges(row_shared_edges, name, folder)

    
    
    pp.close()


In [48]:
folder = "Test"
dfs_sets ={"Nominal Rates":[nr_11, nr_12, nr_15],
           "Nominal Differenced Rates":[ndr_13, ndr_14, ndr_16],
           "Real Rates":[rr_21, rr_22, rr_25],
           "Real Differenced Rates":[rdr_23, rdr_24, rdr_26]}
for name, df_set in dfs_sets.items():
    all_results(df_set, name, folder, show = False)

Working for n conditional variables: 3:  60%|█████████████████████████▊                 | 3/5 [00:00<00:00, 181.76it/s]


Working for n conditional variables: 5: 100%|████████████████████████████████████████████| 5/5 [00:00<00:00, 43.71it/s]


plot_df1 = nominal rates

plot_df2 = nominal rates without M4 variables

plot_df3 = nominal rates without unemployment

plot_df4 = differenced nominal rates

plot_df5 = differenced nominal rates without M4 variables

plot_df6 = differenced nominal rates without unemployment

plot_df7 = real rates

plot_df8 = real rates without M4 variables

plot_df9 = real rates without unemployment

plot_df10 = real differenced rates

plot_df11 = real differenced rates without M4 variables

plot_df12 = real differenced rates without unemployment

In [49]:
# def full_dags(dataframes, name, folder_path):
#     with PdfPages(folder_path + f'/{name}.pdf') as pp:
#         for i, plot_df in enumerate(dataframes):
#             c = PC(plot_df)
#             max_cond_vars = len(plot_df.keys()) - 2
#             model = c.estimate(return_type='pdag', variant='parallel', significance_level=0.01,
#                                max_cond_vars=max_cond_vars, ci_test='pearsonr')
#             edges = model.edges

#             # Plot the DAG
#             graph_DAG(edges, plot_df, title=f'Directed Acyclic Graph {i+1}')
            
#             # Save the plot to the PDF
#             pp.savefig(bbox_inches='tight')

#             # Close the plot to prevent it from being displayed
#             plt.close()
            

In [50]:

# dataframes = [plot_df1, plot_df2, plot_df3, plot_df4, plot_df5, plot_df6, 
#               plot_df7, plot_df8, plot_df9, plot_df10, plot_df11, plot_df12]
# folder_path = 'Test'
# name = 'Full DAGs'
# full_dags(dataframes, name, folder_path)


In [51]:
def full_dags(dataframes, name, folder_path):
    # Create a folder for JPG files if it doesn't exist
    jpg_folder = os.path.join(folder_path, 'jpg')
    os.makedirs(jpg_folder, exist_ok=True)

    with PdfPages(os.path.join(folder_path, f'{name}.pdf')) as pdf:
        for i, plot_df in enumerate(dataframes):
            c = PC(plot_df)
            max_cond_vars = len(plot_df.keys()) - 2
            model = c.estimate(return_type='pdag', variant='parallel', significance_level=0.01,
                               max_cond_vars=max_cond_vars, ci_test='pearsonr')
            edges = model.edges

            # Plot the DAG
            graph_DAG(edges, plot_df, title=f'Directed Acyclic Graph {i+1}')

            # Save the plot to PDF
            pdf.savefig(bbox_inches='tight')
            plt.close()

            # Save the plot to JPG
            plt.figure()
            graph_DAG(edges, plot_df, title=f'Directed Acyclic Graph {i+1}')
            plt.savefig(os.path.join(jpg_folder, f'{name}_{i+1}.jpg'), bbox_inches='tight')
            plt.close()


In [52]:
# Plotting the full dags
dataframes = [plot_df1, plot_df2, plot_df3, plot_df4, plot_df5, plot_df6, 
              plot_df7, plot_df8, plot_df9, plot_df10, plot_df11, plot_df12]
folder_path = 'full dags'
name = 'Full DAGs'
full_dags(dataframes, name, folder_path)


Working for n conditional variables: 4: 100%|████████████████████████████████████████████| 4/4 [00:00<00:00,  5.94it/s]


keep control: Mini
keep control: Util
('M4_y', 'unem') ['Mini', 'Util']
('unem', 'M4 I') []
keep control: Util
keep control: M4_y
('Mini', 'unem') ['Util', 'M4_y']
keep control: Util
('M4 I', 'M4_y') ['Util']
('M4 I', 'Mini') []
keep control: M4 I
('Util', 'M4_y') ['M4 I']
keep control: Mini
keep control: M4_y
('Util', 'unem') ['Mini', 'M4_y']
keep control: Mini
keep control: Util
('M4_y', 'unem') ['Mini', 'Util']
('unem', 'M4 I') []
keep control: Util
keep control: M4_y
('Mini', 'unem') ['Util', 'M4_y']
keep control: Util
('M4 I', 'M4_y') ['Util']
('M4 I', 'Mini') []
keep control: M4 I
('Util', 'M4_y') ['M4 I']
keep control: Mini
keep control: M4_y
('Util', 'unem') ['Mini', 'M4_y']


Working for n conditional variables: 2: 100%|████████████████████████████████████████████| 2/2 [00:00<00:00,  5.99it/s]


keep control: Util
('Mini', 'unem') ['Util']
keep control: Mini
('Util', 'unem') ['Mini']
keep control: Util
('Mini', 'unem') ['Util']
keep control: Mini
('Util', 'unem') ['Mini']


Working for n conditional variables: 3: 100%|████████████████████████████████████████████| 3/3 [00:00<00:00,  6.75it/s]


...

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

In [53]:
# kml3d_ai_level22.columns

In [54]:
plot_df1

Agri      Mini      Util      M4_y      M4 I      unem
FIPS_y  TimePeriod                                                            
1001.0  2002       -0.217648 -0.419451  0.835570  0.055668  0.007415  4.807692
        2003        0.793538  0.191126  0.068072  0.048754  0.008946  5.000000
        2004        0.222678  0.470578  0.505041  0.054306  0.020890  4.715385
        2005        0.169133  0.238805 -0.239593  0.056279  0.032386  3.784615
        2006        0.176881  0.342614  0.436516  0.063506  0.033668  3.315385
...                      ...       ...       ...       ...       ...       ...
56045.0 2016       -0.209103 -0.018912 -0.193847  0.043399  0.003105  5.007692
        2017        0.360970 -0.009520 -0.026597  0.043091  0.006836  4.084615
        2018        0.178369  0.206669  0.065212  0.052718  0.008433  3.538462
        2019       -0.516320  0.209558  0.195492  0.208378  0.002033  2.992308
        2020       -0.494913 -0.334780 -0.095830  0.097720  0.000516  3.869231

[30871 rows x 6 columns]

In [55]:
# def formatted_scatter_matrix(data,fig_len = 15,
#                              c = "C0",
#                             alpha = .1, pp = False):
#         plt.rcParams.update({'font.size': 10})  

#         fig, ax = plt.subplots(figsize = ((fig_len, fig_len)))  
#         pd.plotting.scatter_matrix(data, color = c, 
#                            alpha = .2, 
#                            s = 200 / len(data.keys()), ax=ax)  
#         plt.show()

In [56]:
# plot_df = kml3d_ai_level22
# clusters = plot_df["clusters"].unique()
# plot_df["Cluster Color"] = ""
# for i in clusters:
#     plot_df["Cluster Color"][plot_df["clusters"] == i] = f"C{i}" 
# plot_df

In [57]:
# formatted_scatter_matrix(plot_df, c =plot_df["Cluster Color"],fig_len = 15)

In [58]:
# nr1_df = nr_11
# nr2_df = nr_12
# nr3_df = nr_13
# nr4_df = nr_14
# nr5_df = rr_21
# nr6_df = rr_22
# nr7_df = rr_23
# nr8_df = rr_24

In [59]:
# dfs = [nr1_df, nr2_df, nr3_df, nr4_df, nr5_df, nr6_df, nr7_df, nr8_df]

# for df in dfs:
#     clusters = df["clusters"].unique()
#     df["Cluster Color"] = ""
#     for i in clusters:
#         df.loc[df["clusters"] == i, "Cluster Color"] = f"C{i}"


In [60]:
# formatted_scatter_matrix(nr1_df, c =nr1_df["Cluster Color"],fig_len = 15)

In [61]:
# formatted_scatter_matrix(nr2_df, c =nr1_df["Cluster Color"],fig_len = 15)

In [62]:
# def formatted_scatter_matrixv1(data, fig_len=15, colors=None, alpha=0.1, pp=False):
#     plt.rcParams.update({'font.size': 10})  

#     if colors is None:
#         colors = ["C0"] * len(data)

#     fig, ax = plt.subplots(figsize=(fig_len, fig_len))  
#     pd.plotting.scatter_matrix(data, color=colors, alpha=alpha, s=200 / len(data.keys()), ax=ax)  
#     plt.show()

# # Assuming nr1_df["Cluster Color"] contains the colors for each cluster
# formatted_scatter_matrixv1(nr1_df, fig_len=15, colors=nr1_df["Cluster Color"])


In [63]:
# formatted_scatter_matrix(nr3_df, c =nr3_df["Cluster Color"],fig_len = 15)

In [64]:
# formatted_scatter_matrix(nr4_df, c =nr4_df["Cluster Color"],fig_len = 15)

In [65]:
# formatted_scatter_matrix(nr5_df, c =nr5_df["Cluster Color"],fig_len = 15)

In [66]:
# formatted_scatter_matrix(nr6_df, c =nr6_df["Cluster Color"],fig_len = 15)

In [67]:
# formatted_scatter_matrix(nr7_df, c =nr7_df["Cluster Color"],fig_len = 15)

In [68]:
# formatted_scatter_matrix(nr8_df, c =nr8_df["Cluster Color"],fig_len = 15)

In [69]:
# plot_df1["Cluster"] = np.nan
# plot_df1

In [70]:
# fig,ax = plt.subplots(figsize = (8,4))
# kml3d_ai_level22.plot.scatter(x = "Agri", y = "Util", c = "Mini", cmap = "viridis", s = 5, ax = ax)
# ax.set_title("$Agri \\leftarrow RM4_ \\rightarrow Mini$")
# for i in range(-5,5): ax.axhline(i, c = "k")

In [71]:
# fig,ax = plt.subplots(figsize = (8,4))
# kml3d_ai_level22.plot.scatter(x = "RM4_", y = "RM4I", c = "unem", cmap = "viridis", s = 5, ax = ax)
# ax.set_title("$RM4_ \\rightarrow RM4I \\rightarrow unem$")
# for i in range(-10,2): ax.axhline(i, c = "k")

In [72]:
# fig,ax = plt.subplots(figsize = (8,4))
# kml3d_ai_level22.plot.scatter(x = "Util", y = "RM4_", c = "Agri", cmap = "viridis", s = 5, ax = ax)
# ax.set_title("$Util \\leftarrow RM4_ \\rightarrow Agri$")
# for i in range(-10,2): ax.axhline(i, c = "k")

In [73]:
# fig,ax = plt.subplots(figsize = (8,4))
# kml3d_ai_level22.plot.scatter(x = "Agri", y = "RM4_", c = "RM4I", cmap = "viridis", s = 5, ax = ax)
# ax.set_title("$Agri \\leftarrow RM4_ \\rightarrow RM4I$")
# for i in range(-10,2): ax.axhline(i, c = "k")

In [74]:
# fig,ax = plt.subplots(figsize = (8,4))
# kml3d_ai_level22.plot.scatter(x = "Util", y = "RM4_", c = "RM4I", cmap = "viridis", s = 5, ax = ax)
# ax.set_title("$Util \\leftarrow RM4_ \\rightarrow RM4I$")
# for i in range(-10,2): ax.axhline(i, c = "k")